In [8]:
from selenium import webdriver
import requests
import time
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import pymysql
import re
from lxml import html

#取得餐廳資訊
def getResta(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    print(url)
                         
    # name
    try:
        global name
        for ele in soup.select('h1.restaurants-detail-top-info-TopInfo__restaurantName--1IKBe'):
            name = ele.text
                         
    except AttributeError:
        pass
    
    # city_name
    try:
        p = requests.get(url)
        tree = html.fromstring(p.content)
        cn = tree.xpath('//*[@id="taplc_trip_planner_breadcrumbs_0"]/ul/li[3]/a/span/text()')
        for i in range(len(cn)):
            c_name = cn[i]
                         
    except AttributrError:
        c_name = "None"
                         
    # address
    try:
        global address
        for i in soup.select('span.restaurants-detail-overview-cards-LocationOverviewCard__detailLinkText--co3ei'):
            address = i.text
                         
    except AttributeError:
        address = "None"
                         
    # rate
    try:
        score = soup.find('div', 'ratingContainer')
        rate = score.span['alt']
        Rate = re.findall(r'[\d\.\d]+', rate)
        rate = Rate[0]
                         
    except AttributeError:
        rate = 0
                         
    # comment
    try:
        comment = soup.find('span', 'reviewCount')
        c_comment = comment.get_text()
        Com = re.findall(r'[\d\,\d]+', c_comment)
        c_comment = int(Com[0].replace(",", ""))
                         
    except AttributeError:
        c_comment = 0
                         
    new_id = get_newID()
    href = url
    data_type = "restaurant"
    color = "purple"
    shape = "circle"
    # id,name,city,address,type,comment,rate,href,color,shape
    add_Data(new_id, name, c_name, address,
             data_type, c_comment, rate, href, color, shape)
    
    print("餐廳名：" + name)
print("="*100)


def get_newID():
    last_id = ""
    resta_id = "SELECT MAX(id) FROM resta_data"
    cur.execute(resta_id)
    for id in cur:
        num = re.findall(r'[\d\.\d]+', id[0])
        num = int(num[0]) + 1
        last_id = 'R{0:04}'.format(num)
    return last_id

#加入site資料到資料庫
def add_Data(id, name, city_name, address, type, c_cmt, rate, href, color, shape):
    add_data = ("INSERT INTO resta_data"
                "(id,name,city_name,address,type,comment,rate,href,color,shape) "
                "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)")
    data = (id, name, city_name, address, type, c_cmt, rate, href, color, shape)
    cur.execute(add_data, data)
    # cnx.commit()


#Main
tStart = time.time()
# driver = webdriver.Chrome("C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")


cnx = pymysql.connect(
    host="localhost",
    user="root",
    passwd="12345",
    database='test'
)

cur = cnx.cursor()
cursor = cnx.cursor()
ch_cur = cnx.cursor()

# url = "https://www.tripadvisor.com.tw/Restaurant_Review-g13806900-d4600419-Reviews-Sunny_Hills-Songshan_Taipei.html"
hr = "SELECT near_href,near_site FROM comment_relationship"
cursor.execute(hr)

rhref = []
for j in cursor.fetchall():
    # near
    url = j[0]
    check = url[31:41]
    
    if check == "Restaurant":
        rhref.append(url)

for row in rhref:
    rest_check = "SELECT id FROM resta_data WHERE href = '" + row + "'"
    ch_cur.execute(rest_check)
    entry = ch_cur.fetchone()
    if entry is None:
        getResta(row)
            
tEnd = time.time()
print("Run Time：", tEnd - tStart)

Run Time： 3.5843710899353027
